<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import h3
from shapely.geometry import Polygon
from sqlalchemy import create_engine
import numpy as np
import json
import folium

In [2]:
driver = 'postgresql+psycopg2://'
user = 'govnanyan'
password = ''
hostname = 'dwh-redshift-cluster-1.cirphikau7du.eu-west-1.redshift.amazonaws.com:5439'
db_name = 'dev'
engine = create_engine('{driver}{user}:{password}@{hostname}/{db_name}'.format(
                                                                                driver = driver,
                                                                                user = user,
                                                                                password = password,
                                                                                hostname = hostname,
                                                                                db_name = db_name)
)

In [3]:
trips = pd.read_sql('''
                    select "date",
                    region_id,
                    extract(hour from started_at) as "hour",
                    lat,
                    lon,
                    speed
            from iot_core_stg.track_trip_device	       
            where   ri < 10
                    and "date"::date in ('2023-08-04', '2023-08-17', '2023-08-10', '2023-08-18', '2023-08-07')
                    and extract(hour from started_at) in (8, 18)
                    and region_id in (6,7,11,15,27)
                    and speed between 0 and 28
                    and lat > 0
                    and lon > 0
                 ''', con=engine)
# speed between 5 and 30
# wheel between 5 and 30
trips.head()

,date,region_id,hour,lat,lon,speed
0,2023-08-07,11,18,59.9327,30.3154,0.40
1,2023-08-07,6,18,55.7361,37.4937,0.00
2,2023-08-07,6,18,55.8736,37.6746,6.02
3,2023-08-07,11,18,59.8794,30.4608,13.29
4,2023-08-07,6,18,55.8467,37.6894,23.82


In [22]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29743695 entries, 0 to 29743694
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   date       object 
 1   region_id  int64  
 2   hour       int64  
 3   lat        float64
 4   lon        float64
 5   speed      float64
 6   h3         object 
dtypes: float64(3), int64(2), object(2)
memory usage: 1.6+ GB


In [13]:
h3_level = 10
 
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row['lat'], row['lon'], h3_level)
 
trips['h3'] = trips.apply(lat_lng_to_h3, axis=1)
trips.head()

,date,region_id,hour,lat,lon,speed,h3
0,2023-08-07,11,18,59.9327,30.3154,0.40,8a1106064087fff
1,2023-08-07,6,18,55.7361,37.4937,0.00,8a11aa7a421ffff
2,2023-08-07,6,18,55.8736,37.6746,6.02,8a11aa6219b7fff
3,2023-08-07,11,18,59.8794,30.4608,13.29,8a11060556f7fff
4,2023-08-07,6,18,55.8467,37.6894,23.82,8a11aa623da7fff


In [14]:
trips_speed = trips.groupby(['region_id', 'hour', 'h3']).agg(
    {'speed':['median', 'mean', 'count']}
).reset_index()
trips_speed.columns = ['region_id', 'hour', 'h3', 'median', 'mean', 'count']
trips_speed.head()

,region_id,hour,h3,median,mean,count
0,6,8,8a1181384cb7fff,25.29,25.290000,1
1,6,8,8a1181384d9ffff,14.15,14.369375,48
2,6,8,8a1181a4924ffff,21.25,20.492000,5
3,6,8,8a1181a49257fff,16.45,12.954615,26
4,6,8,8a1181a4925ffff,20.08,19.618571,7


In [15]:
def add_geometry(row):
    points = h3.h3_to_geo_boundary(row['h3'], True)
    return Polygon(points)

trips_speed['geometry'] = trips_speed.apply(add_geometry, axis=1)
trips_speed = gpd.GeoDataFrame(trips_speed, crs='EPSG:4326')
trips_speed.head()

,region_id,hour,h3,median,mean,count,geometry
0,6,8,8a1181384cb7fff,25.29,25.290000,1,"POLYGON ((38.19551 55.11824, 38.19613 55.11767..."
1,6,8,8a1181384d9ffff,14.15,14.369375,48,"POLYGON ((38.19378 55.11875, 38.19439 55.11818..."
2,6,8,8a1181a4924ffff,21.25,20.492000,5,"POLYGON ((38.07006 55.60911, 38.07068 55.60854..."
3,6,8,8a1181a49257fff,16.45,12.954615,26,"POLYGON ((38.06854 55.60723, 38.06916 55.60666..."
4,6,8,8a1181a4925ffff,20.08,19.618571,7,"POLYGON ((38.07018 55.60791, 38.07080 55.60735..."


In [8]:
# m = folium.Map(location=[55.755864, 37.617698], zoom_start=10, tiles='cartodbpositron')
# gdf = gpd.GeoSeries(trips_mean_speed['geometry']).to_json()
# folium.GeoJson(gdf, name = 'boundary').add_to(m)
# m

In [18]:
trips_speed.to_file(r'/Users/georgy/qgis/Ad-Hoc/Cкорость по гео.shp')

In [19]:
trips_speed.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 206320 entries, 0 to 206319
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   region_id  206320 non-null  int64   
 1   hour       206320 non-null  int64   
 2   h3         206320 non-null  object  
 3   median     206320 non-null  float64 
 4   mean       206320 non-null  float64 
 5   count      206320 non-null  int64   
 6   geometry   206320 non-null  geometry
dtypes: float64(2), geometry(1), int64(3), object(1)
memory usage: 11.0+ MB


In [20]:
trips_speed.to_csv(r'/Users/georgy/qgis/Ad-Hoc/Cкорость по гео.csv')

In [21]:
trips_speed.query("h3=='8a10c229b90ffff'")

,region_id,hour,h3,median,mean,count,geometry
187414,15,8,8a10c229b90ffff,18.84,17.618598,107,"POLYGON ((60.58271 56.82745, 60.58307 56.82676..."
193078,15,18,8a10c229b90ffff,21.31,21.119882,85,"POLYGON ((60.58271 56.82745, 60.58307 56.82676..."
